# 1.5 表单flask-wtf

浏览器对面的用户是如何与网站交互呢?一般有几种形式,最常用的就是表单了.表单是前端概念, 也就是说其实和flask本身关系不大,但因为有一个插件--[flask-wtf](http://docs.jinkan.org/docs/flask-wtf/)的存在,表单使用变得异常方便快捷.要了解更多,最好看看[wtf的文档](http://pythonhosted.org/WTForms/)

## flask-wtf

这个名字不是我取的...应该是作者的恶趣味,flask-wtf的安装可以用pip

    pip install flask-wtf

## 跨站请求伪造保护

flask-wtf可以通过给app设置一个密码来防止CSRF(跨站请求攻击)

helloworld.py:

    #...
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'hard to guess string'
    #...
    
    
    
## 定义表单类:

    #...
    from flask.ext.wtf import Form
    from wtforms import StringField, SubmitField from wtforms.validators import Required
    class NameForm(Form):
        name = StringField('What is your name?', validators=[Required()])
        submit = SubmitField('Submit')
    #...
    
    
可以看到定义一个表单基本流程是:

1. 创建一个继承Form的类
2. 定义字段类型变量
3. 设定验证函数


WTForms支持的HTML标准字段:

字段类型|说明
---|---
StringField |文本字段
TextAreaField |多行文本字段
PasswordField |密码文本字段
HiddenField |隐藏文本字段
DateField |文本字段,值为 datetime.date 格式
DateTimeField |文本字段,值为 datetime.datetime 格式
IntegerField |文本字段,值为整数
DecimalField |文本字段,值为 decimal.Decimal
FloatField |文本字段,值为浮点数
BooleanField |复选框,值为 True 和 False
RadioField | 一组单选框
SelectField | 下拉列表
SelectMultipleField |下拉列表,可选择多个值
FileField |文件上传字段
SubmitField |表单提交按钮
FormField|把表单作为字段嵌入另一个表单
FieldList|一组指定类型的字段

内置的验证函数:

验证函数|说明
---|---
Email |验证电子邮件地址
EqualTo |比较两个字段的值;常用于要求输入两次密码进行确认的情况 IPAddress 验证 IPv4 网络地址
Length |验证输入字符串的长度
NumberRange |验证输入的值在数字范围内
Optional |无输入值时跳过其他验证函数
Required |确保字段中有数据
Regexp |使用正则表达式验证输入值
URL |验证 URL
AnyOf |确保输入值在可选值列表中
NoneOf |确保输入值不在可选值列表中




## 使用flask-bootstrap的wtf扩展渲染html:

我们拿index做实验:

In [11]:
%%writefile ../codes/helloworld/templates/myapp/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}hello world!{% endblock %}
{% block content %}
    <div class="container">
        <div class="page-header">
            <p>time:{{ count }}</p>
            <h1>Hello, world! - flask</h1>
        </div>
        <p>answer={{answer}}</p>
        <p>this document carries a cookie </p>
        <p>money={{money|RMBtoUSD}}</p>
        <p>this document carries a cookie </p>
        {{ wtf.quick_form(moneyform,form_type='horizontal') }}
        <img src="{{ url_for('static', filename = 'jftw.jpg') }}"></img>
    </div>    
    {{ super() }}
{% endblock %}

Overwriting ../codes/helloworld/templates/myapp/index.html


其中用到了一个宏`wtf.quick_form`,它的参数是这样的:

    quick_form(form, action=".", method="post", extra_classes=None, role="form", form_type="basic", horizontal_columns=('lg', 2, 10), enctype=None, button_map={}, id="")

参数说明:

    form – The form to output.
    method – <form> method attribute.
    extra_classes – The classes to add to the <form>.
    role – <form> role attribute.
    form_type – One of basic, inline or horizontal. See the Bootstrap docs for details on different form layouts.
    horizontal_columns – When using the horizontal layout, layout forms like this. Must be a 3-tuple of (column-type, left-column-size, right-colum-size).
    enctype – <form> enctype attribute. If None, will automatically be set to multipart/form-data if a FileField is present in the form.
    button_map – A dictionary, mapping button field names to names such as primary, danger or success. Buttons not found in the button_map will use the default type of button.
    id – The <form> id attribute.


In [30]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template
from flask.ext.script import Manager
from flask.ext.bootstrap import Bootstrap
import time

from flask.ext.wtf import Form
from wtforms import StringField, SubmitField 
from wtforms.validators import Required

app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'# 新增
app.debug = True
bootstrap = Bootstrap(app)
manager = Manager(app)

#新增
class MoneyForm(Form):
    money = StringField('How much RMB to change?', validators=[Required()])
    submit = SubmitField('Submit')

@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    return "$"+str(round(float(text)*0.1520,2))


@app.before_request
def before_request():
        g.time = time.asctime()
        if request.cookies.get('answer'):
            g.answer = request.cookies.get('answer')
        else:
            g.answer = "0"
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
#需要注册http方法
@app.route('/',methods = ["GET","POST"])
def hello():
    ##修改
    money = 0
    moneyform = MoneyForm()
    if moneyform.validate_on_submit():
        money = moneyform.money.data
        if moneyform.money.data:
            money = str(moneyform.money.data)
        
        moneyform.money.data =""
    response = make_response(render_template('myapp/index.html',count=g.time,moneyform=moneyform,answer=g.answer,money=money))
    response.set_cookie('answer', money)
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


In [31]:
!python3 ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 17:54:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:28] "GET /static/jftw.jpg HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2016 17:54:28] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:32] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:38] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:54:44] "GET /favicon.ico HTTP/1.1" 200 -
^C


## 更通用的html表单渲染:

`quick_form()`虽然简单好用,但定制性还是不太高,我们来手动实现一个吧

In [22]:
%%writefile ../codes/helloworld/templates/myapp/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}hello world!{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <p>time:{{ count }}</p>
            <h1>Hello, world! - {{name}}</h1>
        </div>
        <p>answer={{answer}}</p>
        <p>money={{money|RMBtoUSD}}</p>
        <p>this document carries a cookie </p>
        <form method="POST" action="/" class = "form-inline">
            {{ moneyform.csrf_token }}
            <div class="form-group">
                {{ moneyform.money.label(for=moneyform.money.id)|safe  }} 
                {{ moneyform.money(size=20,class="form-control") }}
            </div>
            <input type="submit" value="提交" class="btn btn-default">
        </form>
        <img src="{{ url_for('static', filename = 'jftw.jpg') }}"></img>
    </div>
        {{ super() }}
{% endblock %}

Overwriting ../codes/helloworld/templates/myapp/index.html


In [29]:
!python3 ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 17:33:52] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:33:52] "GET /static/jftw.jpg HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2016 17:33:52] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:33:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:33:56] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:34:00] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:34:00] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:34:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 17:34:23] "GET /favicon.ico HTTP/1.1" 200 -
^C


## 用户会话

以上的程序有个问题就是无法记录用户上次的信息,使用session可以做到这点

    from flask import xxx...,session,url_for

    #...
    @app.route('/',methods=["GET","POST"])
    def hello():
        nameform = NameForm()
        if nameform.validate_on_submit():
            session['name'] = nameform.name.data
            if nameform.money.data:
                session['money']  = str(nameform.money.data)
            else:session['money']  = "0"
            return redirect(url_for('hello'))
        if session.get('money'):
            money = session.get('money')
        else:
            money ="0"
        if session.get('name'):
            name = session.get('name')
        else:
            name ="flask"

        tem = render_template("myapp/index.html",
        count=g.time,nameform=nameform,name=name,money=money)se
        return tem
    #...


## flash消息:

请求完成后,有时需要让用户知道状态发生了变化。这里可以使用确认消息、警告或者错 误提醒。一个典型例子是,用户提交了有一项错误的登录表单后,服务器发回的响应重新 渲染了登录表单,并在表单上面显示一个消息,提示用户用户名或密码错误。
这种功能是 Flask 的核心特性



>我们来实现下session和flash

In [78]:
%%writefile ../codes/helloworld/helloworld.py
#coding:UTF-8
from flask import Flask,request,g,make_response,abort,redirect,url_for,render_template,flash,session
from flask.ext.script import Manager
from flask.ext.bootstrap import Bootstrap
import time

from flask.ext.wtf import Form
from wtforms import StringField, SubmitField 
from wtforms.validators import Required

app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'# 新增
app.debug = True
bootstrap = Bootstrap(app)
manager = Manager(app)

#新增
class MoneyForm(Form):
    money = StringField('How much RMB to change?', validators=[Required()])
    submit = SubmitField('Submit')

@app.template_filter('RMBtoUSD')
def RMBtoUSD(text):
    return "$"+str(round(float(text)*0.1520,2))


@app.before_request
def before_request():
        g.time = time.asctime()
#删除一些
@app.errorhandler(404)
def page_not_found(error):
    return "404,page not found!",404,{"a":"af"}
#需要注册http方法
@app.route('/',methods = ["GET","POST"])
def hello():
    ##修改
    old_money = "0"
    money = "0"
    moneyform = MoneyForm()
    if moneyform.validate_on_submit():
        old_money = session.get('money')
        
        if moneyform.money.data:
            money = str(moneyform.money.data)
            session["money"]=money
        if money == "0":
            flash("Looks like you entered 0")
            session["money"]=money
            return redirect(url_for('hello'))                   
        moneyform.money.data =""
        
    response = make_response(render_template('myapp/index.html',count=g.time,
                                             moneyform=moneyform,
                                             answer=old_money+"元",money=money))
    response
    return response

@app.route('/<name>')
def name(name):
    return "time:{count} Hello, world! - {name}".format(count=g.time,name=name)

@app.route("/info")
def info():
    user_agent = request.headers.get('User-Agent')
    return '<p>time:{count}</p><p>Your browser is {agent}</p>'.format(count=g.time,agent=user_agent)

@app.route('/login')
def login():
    abort(404)
@app.route('/infos')
def infos():
    return redirect(url_for('info'))

Overwriting ../codes/helloworld/helloworld.py


In [79]:
%%writefile ../codes/helloworld/templates/myapp/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}hello world!{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <p>time:{{ count }}</p>
            <h1>Hello, world! - {{name}}</h1>
        </div>
        <p>上一次的输入={{answer}}</p>
        {% for message in get_flashed_messages() %}
        
            <div class="alert alert-warning">
                  <button type="button" class="close" data-dismiss="alert">&times;</button>
                  {{ message }}
            </div>
        {% endfor %}
        <p>可以换成:{{money|RMBtoUSD}}</p>
        <form method="POST" action="/" class = "form-inline">
            {{ moneyform.csrf_token }}
            <div class="form-group">
                {{ moneyform.money.label(for=moneyform.money.id)|safe  }} 
                {{ moneyform.money(size=20,class="form-control") }}
            </div>
            <input type="submit" value="提交" class="btn btn-default">
        </form>
        <img src="{{ url_for('static', filename = 'jftw.jpg') }}"></img>
    </div>
        {{ super() }}
{% endblock %}

Overwriting ../codes/helloworld/templates/myapp/index.html


In [80]:
!python3 ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [19/Jan/2016 21:09:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:19] "GET /static/jftw.jpg HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2016 21:09:19] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:22] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/Jan/2016 21:09:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:22] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:28] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:31] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:34] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/Jan/2016 21:09:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2016 21:09:34] "GET /favicon.ico HTTP/1.1" 200 -
127.

## 总结

这块主要是前端技术,用wtf创建表单,用会话+重新定向达到纪录状态外带刷新页面的效果,用flash消息做提示

目前用到的包:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单
